In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("final-project-playfair.ipynb")

# Final Project: Playfair cipher

This choice for the final project will focus on the [Playfair cipher](https://en.wikipedia.org/wiki/Playfair_cipher).

From Wikipedia:
>The technique encrypts pairs of letters (bigrams or digrams), instead of single letters as in the simple substitution cipher and rather more complex Vigenère cipher systems then in use. The Playfair is thus significantly harder to break since the frequency analysis used for simple substitution ciphers does not work with it. The frequency analysis of bigrams is possible, but considerably more difficult. With 600 possible bigrams rather than the 26 possible monograms (single symbols, usually letters in this context), a considerably larger cipher text is required in order to be useful.

You will investigate how to create the key and implement the encryption / decryption algorithms in Python.

## Import your toolkit

You are provided the following functions to help complete this assignment:
* `text_clean`
* `text_block`

Run the cell below to import these assignments

In [ ]:
from finaltoolkit import text_clean, text_block

## The Polybius square

The Playfair cipher uses a Polybius square as the substitution key, similar to the ADFGVX and bifid ciphers.

From Wikipedia:

>The Polybius square, also known as the Polybius checkerboard, is a device invented by the ancient Greeks Cleoxenus and Democleitus, and made famous by the historian and scholar Polybius. The device is used for fractionating plaintext characters so that they can be represented by a smaller set of symbols, which is useful for telegraphy, steganography, and cryptography. The device was originally used for fire signalling, allowing for the coded transmission of any message, not just a finite amount of predetermined options as was the convention before.

To create a Polybius square, first create an empty 5x5 grid. Then, fill it with each of the 26 English characters, with `I` and `J` sharing a square (`I` is only shown when displayed). There are $25!$ different ways to fill the square, which makes it very unlikely that someone would be able to guess your key at random.

In practice, however, a keyword is often used to create the square. The letters of the key (without duplicates) are placed from left to right starting in the top row, moving from top to bottom. When the letters in the keyword are used up, the rest of the alphabet follows in regular order. 

For example, the keyword `playfair example` would result in square below, where duplicate letters are omitted from the square, but still shown in red:

![](https://upload.wikimedia.org/wikipedia/commons/e/ef/Playfair_Cipher_building_grid_omitted_letters.png)

For this assignment, the polybius square must be represented as a string, which is constructed by reading the square left to right, top to bottom.

For example, polybius square from the keyword `playfair example` would be represented as:

In [ ]:
example_keyword = 'PLAYFIREXMBCDGHKNOQSTUVWZ'
example_keyword

**Note**: When creating keys, only the character `I` was stored to the list, not `I/J`. For this assignment, both `I` and `J` should be treated as the character `I`. This will create some odd plaintext messages when deciphering, but humans are good at reading `I`'s as `J`'s when needed. For example, the phrase "Now stop and give me `10 Iumping Iacks`!" would likely be readable to most people.

### Question 1: Accessing elements of the key
The operations needed to encipher and decipher messages require knowing the row and column number for each character in the key. Write a function `get_character` which returns the character at an indicated row and column in the square. You will need to determine how to use the values for `row` and `col` to compute a corresponding index value in the string `key`.

**Note:** The top row will be numbered as row 0, and the last row will be numbered as row 4. The same numbering system applies to the columns.

**Sample Test Cases:**

```python
>> get_character('PLAYFIREXMBCDGHKNOQSTUVWZ', row=0, col=0)
'P'

>> get_character('PLAYFIREXMBCDGHKNOQSTUVWZ', row=1, col=4)
'M'

>> get_character('PLAYFIREXMBCDGHKNOQSTUVWZ', row=4, col=3)
'W'
```

In [ ]:
def get_character(key, row, col):
    ...

In [ ]:
grader.check("q1")

### Question 2

It is also required to determine the row and column number for a given character. Write the functions `get_row` and `get_col` that return an integer that represents the row and column numbers respectively when provided a key and a character contained in the key.

#### Part A: `get_row`

**Sample Test Cases:**

```python
>> get_row('PLAYFIREXMBCDGHKNOQSTUVWZ', 'P')
0

>> get_row('PLAYFIREXMBCDGHKNOQSTUVWZ', 'M')
1

>> get_row('PLAYFIREXMBCDGHKNOQSTUVWZ', 'W')
4
```

In [ ]:
def get_row(key, char):
    ...

In [ ]:
grader.check("q2a")

#### Part B: `get_col`

**Sample Test Cases:**

```python
>> get_col('PLAYFIREXMBCDGHKNOQSTUVWZ', 'P')
0

>> get_col('PLAYFIREXMBCDGHKNOQSTUVWZ', 'M')
4

>> get_col('PLAYFIREXMBCDGHKNOQSTUVWZ', 'W')
3
```

In [ ]:
def get_col(key, char):
    ...

In [ ]:
grader.check("q2b")

### `in` and `not in`

Don't forget that Python has operators `in` and `not in` which can be used to determine if a character is already in a given string.

For example:

In [ ]:
'Y' in 'PLAYFAIR'

In [ ]:
'L' not in 'PLAYFAIR'

This could be helpful if you were to implement a strategy that:
* constructs a string that contains all of the characters in `keyword` AND `LETTERS`
* replaces `J` with `I` 
* creates the key by appending each character from the new string to the key, but only after checking to make sure it's not already in the key

This should guarantee that the key only contains the 25 valid characters. Make sure they appear in the correct order!

### Question 3: Creating the key

Write a function, `keygen` that creates a Polybius square key represented by a string of 25 characters, as seen in the previous examples. 

`keygen` should:
* accept a string for the keyword, named `keyword`. If `keyword` is not provided, then the square should should be filled with the alphabet in normal order.
* An alphabet `LETTERS`, whose default value does not include the character `'J'` since it's not a valid character in Playfair

**Sample Test Cases**:

```python
>> keygen()
'ABCDEFGHIKLMNOPQRSTUVWXYZ'

>> keygen( 'Polybius Cipher' )
'POLYBIUSCHERADFGKMNQTVWXZ'

>> keygen( keyword = 'Long Jump' )
'LONGIUMPABCDEFHKQRSTVWXYZ'
```

In [ ]:
def keygen(keyword='', LETTERS='ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
    
    ...

In [ ]:
grader.check("q3")

## The Playfair Cipher

From Wikipedia:

To encrypt a message, one would break the message into digrams (groups of 2 letters) such that, for example, `HelloWorld` becomes `HE LL OW OR LD`. These digrams will be substituted using the key table. 

Since encryption requires pairs of letters, messages with an odd number of characters usually append an uncommon letter, such as `X`, to complete the final digram. 

The two letters of the digram are considered opposite corners of a rectangle in the key table. To perform the substitution, apply the following 4 rules, in order, to each pair of letters in the plaintext:

1. If both letters are the same (or only one letter is left), add an `X` after the first letter. Encrypt the new pair and continue.
2. If the letters appear on the same row of your table, replace them with the letters to their immediate right respectively (wrapping around to the left side of the row if a letter in the original pair was on the right side of the row). <br>![](https://upload.wikimedia.org/wikipedia/commons/2/29/Playfair_Cipher_10_EX_to_XD.png)
3. If the letters appear on the same column of your table, replace them with the letters immediately below respectively (wrapping around to the top side of the column if a letter in the original pair was on the bottom side of the column).<br>![](https://upload.wikimedia.org/wikipedia/commons/4/44/Playfair_Cipher_02_DE_to_OD.png)
4. If the letters are not on the same row or column, replace them with the letters on the same row respectively but at the other pair of corners of the rectangle defined by the original pair. The order is important – the first letter of the encrypted pair is the one that lies on the same row as the first letter of the plaintext pair.<br>![](https://upload.wikimedia.org/wikipedia/commons/4/40/Playfair_Cipher_01_HI_to_BM.png)

To decrypt, use the inverse (opposite) of the two shift rules, selecting the letter to the left or upwards as appropriate. The last rule remains unchanged, as the transformation switches the selected letters of the rectangle to the opposite diagonal, and a repeat of the transformation returns the selection to its original state. The first rule can only be reversed by dropping any extra instances of the chosen insert letter, `X`, that do not make sense in the final message when finished. This is typically done by hand.

### Question 4: Cleaning the message

When encrypting a message, the Playfair Cipher selects two consecutive plaintext letters and uses the Polybius square to perform a substitution to produce 2 ciphertext characters. The algorithm requires the following:
   1. There should not be any `J`'s in your message. `J` should replaced with `I`
   2. Two consecutive letters must be different characters. If they are the same character, insert an 'X' after the first character to split them up.
   3. The message must be even in length so letters can always be taken in pairs. If it's not even in length after meeting the first two requirements, add an 'X' at the end of the message to make it even in length
   
Write a function `playfair_textclean` that ensures the criteria are met for any provided message.

**Sample Test Cases**:

```python
>>> playfair_textclean( 'HEY' )
'HEYX'

>>> playfair_textclean( 'Jump' )
'IUMP'

>>> playfair_textclean( 'Hide the gold in the tree stump' )
'HIDETHEGOLDINTHETREXESTUMP'

>>> playfair_textclean( 'mississippi' )
'MISXSISXSIPXPI'
```

In [ ]:
def playfair_textclean(message):
    ...

In [ ]:
grader.check("q4")

### Question 5: Encryption / Decryption

At this point you should now be able write a function `playfair` that implements the playfair cipher.

The function should:
* accept a keyword (`str`) and message (`str`)
* an optional boolean value `decrypt` that when set to `True` decrypts the `message` `False` encrypts the `message`
* return a decrypted plaintext as a lowercase string and return an encrypted ciphertext as an uppercase message blocked into character groups with length of 5

  
**Sample Test Cases**:

```python
>>> playfair('playfair example', 'Hide the gold in the tree stump')
'BMODZ BXDNA BEKUD MUIXM MOUVI F'

>>> playfair('playfair example', 'BMODZ BXDNA BEKUD MUIXM MOUVI F', False)
'hidethegoldinthetrexestump'
```

In [ ]:
def playfair(keyword, message, decrypt=False):
    
    ...

In [ ]:
grader.check("q5")

## Ciphertext analysis

There is a file included with this notebook that contains the first chapter of Pride and Prejudice. The questions below will ask you to analyze a ciphertext that uses this chapter as the plaintext. These responses will only be graded for accuracy once you've submitted your assignment, so make sure you are certain of your answers before submitting!

Run the cell below to load the chapter to the string named `plaintext`

In [ ]:
with open('pride-prejudice-chapter-01.txt') as f:
    plaintext = f.read()

<!-- BEGIN QUESTION -->

### Question 6: Creating a bar chart

Encrypt the plaintext using a keyword of `Jane Austen`, then create a bar chart that shows the single character frequencies of each of the 25 English characters used in the message.

In [ ]:
ciphertext = ...


<!-- END QUESTION -->

### Question 7: Index of Coincidence

Write a function `index_of_coincidence` to compute the index of coincidence of the ciphertext.

**Hint:** Reference Lesson 17 and Lab 04 if you need a refresher!

In [ ]:
def index_of_coincidence(message, LETTERS='ABCDEFGHIKLMNOPQRSTUVWXYZ'):
    
    ...

In [ ]:
grader.check("q7")

<!-- BEGIN QUESTION -->

### Question 8: Classifying the cipher

The Playfair cipher is what's known as a polygraphic cipher, which we have not yet covered in this cousre.. It's not a monoalphabetic substitution cipher or a polyalphabetic substitution cipher. However, which of those does it seem to behave like? Describe the reasoning for your answer below, including specific details from your index of coincidence calculation and bar chart to back up your response.

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

In [ ]:
grader.export(pdf=False, force_save=True)